<a href="https://colab.research.google.com/github/KoheiToda/keibaAI001/blob/main/KeibaAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from datetime import date
from typing import List, Dict
from tqdm import tqdm
from pandas.core.frame import DataFrame
import time

In [2]:
def get_url_ids(year_from: str, year_to: str) -> List[str]:
  """
  ネット競馬のレース結果URLに使用する末尾IDを取得する

  Parameters
  ----------
  year_from : str
    開始年 YYYY
  year_to : str
    終了年 YYYY

  Returns
  -------
  url_ids : list[str]
    レース結果を保持するURLの末尾ID
    YYYYXXCCDDRR
    YYYY: 開催年
    XX: レース場（1～10）
      01:札幌 02:函館 03:福島 04:新潟 05:東京 06:中山 07:中京 08:京都 09:阪神 10:小倉
    CC: 何回目　（1～12）
    DD: 何日目　（1～12）
    RR: レース数（1～12）
  """
  REGION_MAX = 10 # レース場は下記を表す
  # 01:札幌 02:函館 03:福島 04:新潟 05:東京 06:中山 07:中京 08:京都 09:阪神 10:小倉
  COUNT_MAX = 12 # 1ヵ月を表すため、最大12回
  DAY_MAX = 12 # その回の何日目の開催か。土日祝の開催のため、多くて12で計算
  RASE_MAX = 12 # 1日のレース数は原則12レース


  results = []
  if not((year_from.isdecimal()) and len(year_from) == 4):
    return None
  if not((year_to.isdecimal()) and len(year_to) == 4):
    return None
  if year_from > year_to:
    return None

  for year in range(int(year_from), int(year_to) + 1):
    for region in range(1, REGION_MAX + 1):
      for count in range(1, COUNT_MAX + 1):
        for day in range(1, DAY_MAX + 1):
          for rase in range(1, RASE_MAX + 1):
            id = str(year).zfill(4) + str(region).zfill(2) + str(count).zfill(2) + str(day).zfill(2) + str(rase).zfill(2)
            results.append(id)
  return results


In [3]:
def crawing_net_keiba(url_ids: List[str], pre_results: Dict = {}) -> Dict:
  """
  ネット競馬の指定した範囲のurlをクローリングする

  Parameters
  ----------
  url_ids : str
    YYYYXXCCDDRR
    YYYY: 開催年
    XX: レース場（1～10）
    CC: 何回目　（1～12）
    DD: 何日目　（1～12）
    RR: レース数（1～12）

  Returns
  -------
  results : dict
    key: url_id
    value: url先のテーブル情報
  """

  url_head = 'https://db.netkeiba.com/race/'
  results = pre_results
  skip_url = ''

  for url_id in tqdm(url_ids):
    # 既に結果を取得済なら飛ばす
    if url_id in results.keys():
      continue

    # スキップ対象のURLなら飛ばす
    if len(skip_url) > 0 and url_id[0: len(skip_url)] == skip_url:
      continue
    else:
      skip_url =''

    # クローリングを実施する
    url = url_head + url_id + '/'
    try:
      time.sleep(1)
      results[url_id] = pd.read_html(url)

    # レース情報が存在しない場合
    except IndexError:
      if url_id[-2:] != '01':     # RR = 1レース目以外
        skip_url = url_id[0:10]   # YYYYXXCCDD が同じもの飛ばす（それ以降のレースは存在しないため）
        continue
      elif url_id[-4: -2] != '01':  # DD = 1日目以外
        skip_url = url_id[0:8]    # YYYYXXCC が同じものを飛ばす（それ以降の日数は開催していない）
        continue
      elif url_id[-6: -4] != '01': # CC = 1回目以外
        skip_url = url_id[0:6]    # YYYYXX が同じものを飛ばす（その以降の回数は開催していない）
        continue
      else: # CCDDCC = 010101
        skip_url = url_id[0:6]    # YYYYXX が同じものを飛ばす（そのレース場で今年開催していない）
        continue
        
    # それ以外のエラー
    except:
      break
  return results

In [14]:
def get_rase_results(crawing_results: dict, pre_results: DataFrame = None) -> DataFrame:
  """
  indexにurl_idを設定し、1つのDataFrameにして返す
  """
  for key in crawing_results.keys():
    crawing_results[key][0].index = [key] * len(crawing_results[key][0])
  results = pd.concat([crawing_results[key][0] for key in crawing_results.keys()], sort=False)
  return results

In [4]:

url_ids = get_url_ids('2022', '2022')
crawing_results_2022 = crawing_net_keiba(url_ids)

100%|██████████| 17280/17280 [07:25<00:00, 38.78it/s]


In [20]:
rase_results_2022 = get_rase_results(crawing_results_2022)

In [21]:
rase_results_2022.to_pickle('rase_results_2022.pickle')

In [22]:
rase_results_2022_copy = pd.read_pickle('rase_results_2022.pickle')

In [23]:
rase_results_2022_copy

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
202205010101,1,1,2,モネ,牝3,54.0,津村明秀,1:26.3,NaN,7.9,4.0,458(+2),[東] 牧光二
202205010101,2,2,4,ランコントル,牝3,54.0,Ｍ．デム,1:26.5,1,4.2,1.0,448(+6),[東] 矢野英一
202205010101,3,2,3,ジューンポンポン,牝3,54.0,田辺裕信,1:26.5,クビ,5.0,3.0,476(0),[東] 和田勇介
202205010101,4,1,1,エレガントミッシー,牝3,54.0,宮崎北斗,1:27.0,3,11.3,6.0,422(-4),[東] 佐藤吉勝
202205010101,5,4,7,ピカリエ,牝3,54.0,伊藤工真,1:27.1,3/4,4.6,2.0,490(+8),[東] 金成貴史
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202210010612,10,8,14,アイオープナー,牝5,55.0,黛弘人,2:01.5,2,64.8,12.0,432(-2),[東] 宗像義忠
202210010612,11,2,2,メイレンシュタイン,セ4,56.0,丹内祐次,2:01.6,クビ,14.4,6.0,466(+12),[東] 勢司和浩
202210010612,12,7,11,ヒナノコバン,牡5,56.0,秋山稔樹,2:01.6,クビ,42.9,11.0,468(-6),[東] 本間忍
202210010612,13,6,10,ケリアテソーロ,牝4,52.0,藤田菜七,2:01.7,1/2,94.8,14.0,442(-2),[東] 伊坂重信
